In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.conf import SparkConf
spark = SparkSession.builder.appName("Weather") \
        .config("spark.dynamicAllocation.enabled", "false") \
        .config("spark.executor.memory","1000m") \
        .getOrCreate()

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2 pyspark-shell'

In [3]:
schema = StructType([
    StructField('dt', IntegerType()),
    StructField('main.temp', FloatType()),
    StructField('main.pressure', IntegerType()),
    StructField('main.humidity', IntegerType()),
    StructField('visibility', IntegerType()),
    StructField('wind.speed', FloatType()),
    StructField('clouds.all', IntegerType()),
    StructField('rain.3h', FloatType(), False),
    StructField('snow.3h', FloatType(), False),
    StructField('pop', FloatType())
])

In [ ]:
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "instance-tram-1:9092") \
    .option("subscribe", "weather_data_processed") \
    .option("failOnDataLoss", "false") \
    .load()
table = df.selectExpr("CAST(key AS STRING)","CAST(value AS STRING)").select(from_json('value', schema).alias('tmp')).select('tmp.*')
df_kafka = table\
    .withColumn('value', to_json(struct(table["`dt`"], table["`main.temp`"], table["`main.pressure`"], table["`main.humidity`"], table["`visibility`"],
                                       table["`wind.speed`"], table["`clouds.all`"], table["`rain.3h`"], table["`snow.3h`"], table["`pop`"])))
query = df_kafka.writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "instance-tram-1:9092") \
    .option("topic", "weather-aggregates") \
    .option("checkpointLocation", "~/agg_checkpoint/") \
    .start() \
    .awaitTermination()

22/01/16 19:24:44 WARN org.apache.spark.sql.streaming.StreamingQueryManager: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [14]:
spark.stop()